### Análisis preliminar del conjunto de datos

In [11]:
# Importaciones
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import sys
sys.path.append("..")
from src import sp_eda_limpieza as sp

In [2]:
# Leer el archivo
df_raw = pd.read_excel("../data/amazon_churn_datosbrutos.xlsx")

In [3]:
# Hacer una copia del dataframe con el que trabajar
df = df_raw.copy()

In [ ]:
# Hacer un análisis exploratorio de los datos 
sp.eda_preliminar(df)

### Normalización, estandarización y limpieza de datos

In [6]:
df.columns = df.columns.str.replace(" ","_").str.lower() # Pasar los nombres de las columnas a minúsculas y sin espacios
for col in df.select_dtypes(include='O').columns: # Pasar los nombres de los valores de las columnas categóricas a minúsculas y sin espacios
  df[col] = df[col].apply(lambda x: x.replace(" ","_").lower() if isinstance(x, str) else x)

In [7]:
# Validación de datos
df['churn_label_binario'] = df['churn_label'].map({'Yes': 1, 'No':0}) 
df['churn_label_binario'].equals(df['churned']) # Comprobar que 'churn_label' y 'churn' son la misma columna pero con el tipo de dato distinto, por lo que eliminaremos 'churned'.
# Comprobar que la columna 'under_30' y 'senior' se corresponden con la columna 'edad'
validacion_under_30 = df[(df['under_30'] == 'yes') & (df['age'] >= 30)]
errores_under_30_count = validacion_under_30.shape[0]
validacion_senior = df[(df['senior']=='yes') & (df['age'] < 65)]
errores_validacion_senior = validacion_senior.shape[0]

In [8]:
# Eliminar columnas irrelevantes para el objetivo del análisis
columnas_eliminar = ['churned', 'phone_number','hobby','favorite_tv_show', 'internal_notes','churn_label_binario']
df.drop(columns= columnas_eliminar, inplace=True)

In [9]:
# Limpieza de datos
df['contract_type'] = df['contract_type'].str.strip('_').str.replace("-","_", regex=True) # Eliminar espacios y guiones sobrantes de la columna 'contrat_type'

df['monthly_charge'] = df['monthly_charge'].apply(lambda x: float(x.replace("_usd","")) if isinstance(x,str) else x) # Eliminar carácteres sobrantes de 'monthly_charge'

df['unlimited_data_plan'] = df['unlimited_data_plan'].map({1:'yes', 0:'no'}) # Mapear los valores '1' y '0' por 'yes' y 'no' de 'unlimited_data_plan'

df['gender'] = df['gender'].replace('hombre', 'male') # Cambiar los valores 'hombres' por 'male'

df['preferred_contact_method'] = df['preferred_contact_method'].replace('teléfono', 'phone') # Traducción de valores

df['customer_segment'] = df['customer_segment'].replace({'alto':'high','medio':'medium','bajo':'low'}) # Traducción de valores

df['applied_discount'] = df['applied_discount'].map({True:'yes', False:'no'}) # Reemplazar 'true' y 'false' por 'yes' y 'no'

df['contact_date'] = df['contact_date'].astype(str).str.replace("-","/") # Normalización de la fecha de contacto

df['average_monthly_expenses'] = df['average_monthly_expenses'].apply(lambda x: round(x, 2)) # Redondeo de valores en 'average_monthly_expenses'

df = df.rename(columns={'senior':'senior_65'}) # Cambiar el nombe de la columna 'senior' para indicar que se refieren a 65 o más

In [15]:
df.to_excel("../data/datos_churn_limpios.xlsx", index=False)

In [ ]:
# Proxima sesion gestion de nulos y asignacion del tipo de dato, y eliminar duplicados si hay